In [1]:
print('Hello Students, enjoy coding and GPUing...')

Hello Students, enjoy coding and GPUing...


In [2]:
print("Thank you Professor...")

Thank you Professor...


In [3]:
!pip install wandb transformers[torch] huggingface-hub datasets evaluate rouge_score nltk accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-

# Logins

In [4]:
import wandb
wandb.login(key='WANDB key')

import huggingface_hub
huggingface_hub.login(token='HUGGING FACE KEY')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Dataset

In [5]:
from datasets import load_dataset

raw_datasets = load_dataset("allenai/cord19", "fulltext", "abstract",
                            trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/368618 [00:00<?, ? examples/s]

In [6]:
import numpy as np
def select_datapints(dataset):
    dataset['select'] = (len(dataset['fulltext']) > 0) and (len(dataset['abstract']) > 0)
    return dataset

raw_datasets['train'] = raw_datasets['train'].map(select_datapints)
indices_required = np.where(np.array(raw_datasets['train']['select']) == True)[0]
raw_datasets['train'] = raw_datasets['train'].select(indices_required)
raw_datasets

Map:   0%|          | 0/368618 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['cord_uid', 'sha', 'source_x', 'title', 'doi', 'abstract', 'publish_time', 'authors', 'journal', 'url', 'fulltext', 'select'],
        num_rows: 105097
    })
})

In [7]:
raw_datasets = raw_datasets.remove_columns(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'publish_time', 'authors', 'journal', 'url'])

import gc
gc.collect()

from datasets import DatasetDict
train_testvalid = raw_datasets['train'].train_test_split(test_size=0.2)

# Further split the test+validation into test and validation sets evenly
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

# Combine the splits into a single DatasetDict
raw_datasets = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']
})

train_testvalid = test_valid = None
gc.collect()
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['abstract', 'fulltext', 'select'],
        num_rows: 84077
    })
    test: Dataset({
        features: ['abstract', 'fulltext', 'select'],
        num_rows: 10510
    })
    validation: Dataset({
        features: ['abstract', 'fulltext', 'select'],
        num_rows: 10510
    })
})

# Tokenizer

In [8]:
from transformers import AutoTokenizer

model_checkpoint = 'lxyuan/distilbart-finetuned-summarization'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [9]:
max_length = 1024
#max_target_length = 128
prefix = ''

def preprocess_function(examples):
    # inputs = [prefix + doc for doc in examples["fulltext"]]
    model_inputs = tokenizer(examples['fulltext'], max_length=max_length, truncation=True, padding=True, return_tensors='pt')
    # Setup the tokenizer for targets
    labels = tokenizer(
        examples["abstract"], max_length=max_length, truncation=True, padding=True, return_tensors='pt')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True, num_proc=8)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=8):   0%|          | 0/84077 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/10510 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/10510 [00:00<?, ? examples/s]

In [11]:
tokenized_train = tokenized_datasets['train']
tokenized_val = tokenized_datasets['validation']
tokenized_test = tokenized_datasets['test']

# Model

In [12]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/328 [00:00<?, ?B/s]

In [34]:
batch_size = 12
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"distilbart-finetuned-cord-19",

    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",

    learning_rate=2e-5,
    optim="adamw_torch",

    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=64,

    weight_decay=0.01,
    # save_total_limit=2,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    report_to='wandb',
    # push_to_hub=True,
)

In [35]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Metric

In [36]:
import nltk
import numpy as np
from evaluate import load
nltk.download("punkt")

metric = load('rouge')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds
    ]
    decoded_labels = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        use_aggregator=True,
    )
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [37]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [38]:
import torch
torch.cuda.empty_cache()
gc.collect()

140

In [39]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,1.071600,0.805287,34.574700,13.960900,21.367000,31.745800,140.079600


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


TrainOutput(global_step=109, training_loss=1.0715586811030677, metrics={'train_runtime': 5519.2078, 'train_samples_per_second': 15.234, 'train_steps_per_second': 0.02, 'total_flos': 1.2957881650171085e+17, 'train_loss': 1.0715586811030677, 'epoch': 0.9955758527187099})

# Save

In [40]:
trainer.save_model('DistilBART-sumamrization')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [41]:
import shutil


# Specify the directory you want to compress and the output filename
directory_name = './DistilBART-sumamrization'
output_filename = 'DistilBART-sumamrization'

# Create a zip archive of the directory
shutil.make_archive(output_filename, 'zip', directory_name)


# Specify the directory you want to compress and the output filename
directory_name = './distilbart-finetuned-cord-19'
output_filename = 'Distilbart-finetuned-cord-19'

# Create a zip archive of the directory
shutil.make_archive(output_filename, 'zip', directory_name)


'/content/Distilbart-finetuned-cord-19.zip'

# Checking on validation data

In [ ]:
eval_result = trainer.evaluate()
eval_result

# Testing on Test Data

In [42]:
fine_tuned_model = 'DistilBART-sumamrization'
new_model = AutoModelForSeq2SeqLM.from_pretrained(fine_tuned_model)
new_tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model)

In [43]:
trainer2 = Seq2SeqTrainer(
    model = new_model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
    tokenizer=new_tokenizer,
    compute_metrics=compute_metrics,
)

In [44]:
gc.collect()
torch.cuda.empty_cache()

In [52]:
test_result = trainer2.evaluate()
test_result

{'eval_loss': 0.7785233855247498,
 'eval_rouge1': 34.6544,
 'eval_rouge2': 13.9661,
 'eval_rougeL': 21.4553,
 'eval_rougeLsum': 31.791,
 'eval_gen_len': 140.0093,
 'eval_runtime': 3576.5795,
 'eval_samples_per_second': 2.939,
 'eval_steps_per_second': 0.245}